In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta

driver = webdriver.Chrome()

start_date = datetime(2022, 4, 18)  
end_date = datetime(2013, 6, 16)  

data = []

def scrape_table(date):
    try:
        driver.get("https://kalimatimarket.gov.np/price")
        date_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "datePricing"))
        )

        date_input.clear()
        date_input.send_keys(date.strftime("%m-%d-%Y"))

        submit_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-theme.comment-btn"))
    )
        submit_button.click()

        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "commodityPriceParticular"))
        )

        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            cols = [col.text for col in cols]
            if cols:
                cols.append(date.date()) 
                data.append(cols)

    except Exception as e:
        print(f"Error scraping data for {date.date()}: {e}")

current_date = start_date
driver.get("https://kalimatimarket.gov.np/lang/en")
while current_date >= end_date:
    scrape_table(current_date)
    current_date -= timedelta(days=1)

driver.quit()

df = pd.DataFrame(data, columns=["Commodity", "Unit", "Minimum", "Maximum", "Average", "Date"])

print(df)

Error scraping data for 2022-02-09: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF777FE1F52+60322]
	(No symbol) [0x00007FF777F5CEC9]
	(No symbol) [0x00007FF777E17EBA]
	(No symbol) [0x00007FF777E1DCEE]
	(No symbol) [0x00007FF777E20641]
	(No symbol) [0x00007FF777E206E0]
	(No symbol) [0x00007FF777E6733B]
	(No symbol) [0x00007FF777E6773C]
	(No symbol) [0x00007FF777E5B2AC]
	(No symbol) [0x00007FF777E8C25F]
	(No symbol) [0x00007FF777E5ADB6]
	(No symbol) [0x00007FF777E8C430]
	(No symbol) [0x00007FF777EABC80]
	(No symbol) [0x00007FF777E8BFC3]
	(No symbol) [0x00007FF777E59617]
	(No symbol) [0x00007FF777E5A211]
	GetHandleVerifier [0x00007FF7782F94AD+3301629]
	GetHandleVerifier [0x00007FF7783436D3+3605283]
	GetHandleVerifier [0x00007

In [2]:
df.to_csv('missing_data.csv', index=False)

In [4]:
import pandas as pd
df = pd.read_csv('Kalimati_Dataset2.csv')
df.head()

C:\Users\Nawap Bastola\AppData\Local\Temp\ipykernel_19972\1699161221.py:2: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Kalimati_Dataset2.csv')


,Commodity,Date,Minimum,Maximum,Average
0,Tomato Big(Nepali),6/16/2013,35,40,37.5
1,Tomato Small(Local),6/16/2013,26,32,29.0
2,Potato Red,6/16/2013,20,21,20.5
3,Potato White,6/16/2013,15,16,15.5
4,Onion Dry (Indian),6/16/2013,28,30,29.0


In [6]:
df['Date'] = pd.to_datetime(df['Date'])

def convert_to_float(value):
    if isinstance(value, str):
        return float(value.replace('Rs ', ''))
    else:
        return float(value)

df['Minimum'] = df['Minimum'].apply(convert_to_float)
df['Maximum'] = df['Maximum'].apply(convert_to_float)
df['Average'] = df['Average'].apply(convert_to_float)

# Print the DataFrame
print(df)

                  Commodity       Date  Minimum  Maximum  Average
0        Tomato Big(Nepali) 2013-06-16     35.0     40.0     37.5
1       Tomato Small(Local) 2013-06-16     26.0     32.0     29.0
2                Potato Red 2013-06-16     20.0     21.0     20.5
3              Potato White 2013-06-16     15.0     16.0     15.5
4        Onion Dry (Indian) 2013-06-16     28.0     30.0     29.0
...                     ...        ...      ...      ...      ...
305187     Fish Fresh(Rahu) 2024-06-12    320.0    330.0    325.0
305188  Fish Fresh(Bachuwa) 2024-06-12    240.0    250.0    245.0
305189   Fish Fresh(Chhadi) 2024-06-12    240.0    260.0    250.0
305190  Fish Fresh(Mungari) 2024-06-12    320.0    330.0    325.0
305191          King Oyster 2024-06-12    280.0    300.0    290.0

[305192 rows x 5 columns]


In [8]:
df.to_csv('Kalimati_Dataset2.csv', index=False)

In [19]:
d = pd.read_csv('Kalimati_Dataset2.csv')
d.tail()

C:\Users\Nawap Bastola\AppData\Local\Temp\ipykernel_4468\791978599.py:1: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('Kalimati_Dataset2.csv')


,Commodity,Date,Minimum,Maximum,Average
305189,Fish Fresh(Chhadi),6/12/2024,240.0,260.0,250.0
305190,Fish Fresh(Mungari),6/12/2024,320.0,330.0,325.0
305191,King Oyster,6/12/2024,280.0,300.0,290.0
305192,No data available in table,NaN,NaN,NaN,NaN
305193,No data available in table,NaN,NaN,NaN,NaN


In [20]:
d = d.drop(d.index[-2:])
d.to_csv('Kalimati_Dataset2.csv', index=False)

In [ ]:
import pandas as pd
df_train = pd.read_csv('kalimati-tarkari-prices-from-may-2021-to-september-2023.csv',)
df_train.columns = ["Commodity", "Date", "Unit","Minimum", "Maximum", "Average"]  
df_train

In [ ]:
df_train['Date'][40000]

In [ ]:
df_train.describe()

In [ ]:
filtered_df = df_train[["Commodity","Average","Date"]]
filtered_df

In [ ]:
def convert_to_yyyy_mm_dd(date_str):
    try:
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except ValueError:
        return pd.to_datetime(date_str, format='%d/%m/%Y').strftime('%Y-%m-%d')

# Apply the custom function to the mixed date column
filtered_df['Date'] = filtered_df['Date'].apply(convert_to_yyyy_mm_dd)

In [ ]:
filtered_df

In [ ]:
filtered_df['Average'] = filtered_df['Average'].apply(lambda x: float(x.replace('Rs', '')))
filtered_df

In [ ]:
type(filtered_df['Date'][0])

In [ ]:
filtered_df

In [ ]:
filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
filtered_df['Date'][0]

In [ ]:
tomato_df = filtered_df[['Commodity']=="Tomato Big(Indian)"]
filtered_df.head()